In [ ]:
from typing import List, Any, Tuple, Union
import networkx as nx
import matplotlib as mpl
import json
from cdlib import *
from matplotlib import pyplot as plt
import seaborn
from itertools import combinations,permutations
import numpy as np
import pandas as pd
import random
import jieba
from zhon.hanzi import punctuation
#jieba.load_userdict("")
import jieba.posseg as pseg
import opencc
import os
import re
import string
import warnings
import logging
from gensim.models import Word2Vec
from gensim import corpora,models,similarities
from collections import defaultdict,OrderedDict,Counter
from pprint import pprint
from gensim.models.word2vec import LineSentence,PathLineSentences
from gensim.corpora import WikiCorpus
import multiprocessing
from nltk.tokenize import word_tokenize
from nltk.tokenize import  sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
from nltk.stem import LancasterStemmer
from nltk.stem import RegexpStemmer
import time
import community
from wordcloud import WordCloud,STOPWORDS
from PIL import Image
from PIL import ImageColor
from imageio import imread,imsave
from cdlib import evaluation as ev
from cdlib import viz,readwrite,algorithms

from community import modularity
# import matplotlib
# matplotlib.use('TkAgg')
# import compare_algorithms as ca
# plt.style.use('science')
#忽略警告
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#同步更新版本
class NodeOrder():
    def __init__(self,G):
        self.G=G
    def get_adjacency_list(self):
        # 获取邻居节点
        adj_list={}
        for nd in list(self.G.nodes()):
            nei = list(self.G.neighbors(nd))
            adj_list[nd] = nei
        return adj_list
    def default_order(self):
        """#the nodes are processed in the order of their original order in the graph"""

        order=list(self.G.nodes())
        return order
    def random_order(self):
        """a random node-ordering is used"""
        # print(list(self.G.nodes()))
        nodes=list(self.G.nodes())


        random.shuffle(nodes)
        return nodes
    def degree_order(self):
        """nodes are ordering by the degree of node",
        if some node have the same degree,ordering these nodes by its original exists order"""
        order_list = sorted({v: self.G.degree(v) for v in list(self.G.nodes())}.items(), reverse=True,
                            key=lambda k: k[1])
        order =[l[0] for l in order_list]
        c_1 = dict(Counter([c for n, c in self.G.degree()]))
        return order,c_1
    def degree_centrality_order(self):
        """nodes are ordering by the degree centrality of node",
        if some node have the same degree centrality,ordering these nodes by its original exists order"""
        o=nx.degree_centrality(self.G)
        order_list=sorted(o.items() ,reverse=True,key= lambda k:k[1])
        order =[l[0] for l in order_list]
        c_1 = dict(Counter([c for  c in o.values()]))
        return order,c_1
    def information_centrality_order(self):

        """nodes are ordering by the information_centrality of node
        if some node have the same degree, ordering these nodes by its original exists order"""
        o=nx.information_centrality(self.G)
        order_list = sorted(o.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def closeness_centrality_order(self):
        """nodes are ordering by the closeness_centrality of node
        if some node have the same closeness centrality , ordering these nodes by its original exists order"""
        o=nx.closeness_centrality(self.G)
        order_list = sorted(o.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def betweenness_centrality_order(self):
        """nodes are ordering by the betweenness_centrality of node
        if some node have the same betweenness centrality, ordering these nodes by its original exists order"""
        o = nx.betweenness_centrality(self.G)
        order_list = sorted(o.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def eigenvector_centrality_order(self):
        """nodes are ordering by the eigenvector_centrality of node
        if some node have the same eigenvector_centrality , ordering these nodes by its original exists order"""
        o = nx.eigenvector_centrality(self.G)
        order_list = sorted(o.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def katz_centrality_order(self):
        """nodes are ordering by the katz_centrality of node
        if some node have the same katz_centrality, ordering these nodes by its original exists order"""
        o=nx.katz_centrality(self.G)
        order_list = sorted(o.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def leadership_order(self):
        #wei实现
        """nodes are ordering by the leader_ship of node
        if some node have the same leader_ship, ordering these nodes by its original exists order"""
        order_list = sorted(nx.leadership(self.G).items(), reverse=True, key=lambda k: k[1])

        order = [l[0] for l in order_list]
        return order
    def subgraph_centrality_order(self):
        """nodes are ordering by the Alpha of node
        if some node have the same Alpha, ordering these nodes by its original exists order"""
        o = nx.subgraph_centrality(self.G)
        order_list = sorted(o.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def Alpha_order(self):
        #未实现
        """nodes are ordering by the Alpha of node
        if some node have the same Alpha, ordering these nodes by its original exists order"""
        o={}
        order_list = sorted([(v, self.G.degree(v)) for v in self.G.nodes()], reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def PageRank_order(self):
        """nodes are ordering by the PageRank of node
        if some node have the same PageRank, ordering these nodes by its original exists order"""
        o=nx.pagerank(self.G)
        order_list = sorted(o.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in o.values()]))
        return order, c_1
    def Hindex_order(self):
        """nodes are ordering by the Hindex of node
        if some node have the same Hindex, ordering these nodes by its original exists order"
        as this method ,there may be a few node which have the same h-index,so it's not fit to order  nodes"""
        h={}

        for n in list(self.G.nodes()):
            d=list(self.G.degree(list(self.G.neighbors(n))))
            d=sorted(d,key=lambda x:x[1],reverse=True)
            i=1
            for node ,degree in d:
                if i>degree:
                    break
                else:
                    i=i+1
            h[n]=i-1
        order_list = sorted(h.items(), reverse=True, key=lambda k: k[1])
        order = [l[0] for l in order_list]
        c_1 = dict(Counter([c for c in h.values()]))
        return order,c_1
    def count_his(self):
        #需将排序结果存入pandas中
        o1,h1=self.degree_order()
        o2,h2=self.degree_centrality_order()
        # o3,h3=self.information_centrality_order()
        o3, h3 = self.Hindex_order()
        o4,h4=self.closeness_centrality_order()
        o5,h5=self.betweenness_centrality_order()
        o6,h6=self.eigenvector_centrality_order()
        # o7,h7=self.katz_centrality_order()
        o7, h7 =self.degree_order()
        o8,h8=self.subgraph_centrality_order()
        o9,h9=self.PageRank_order()

        H=[h1,h2,h3,h4,h5,h6,h7,h8,h9]
        O=[o1,o2,o3,o4,o5,o6,o7,o8,o9]
        X=[s.keys() for s in H]
        Y=[s.values() for s in H ]
        sp=list(range(331,340,1))
        T=['degree','degree_centrality','Hindex_order','closeness_centrality',\
           'betweenness_centrality','eigenvector_centrality',\
           'katz_centrality','subgraph_centrality','PageRank']


        for i ,t,x,y, o ,h in zip(sp,T,X,Y,O,H):
            print(t+"排序结果")
            print(o)
            print(t+"指标频数统计")
            h=sorted(h.items(),key=lambda x:x[1],reverse=True)
            print(h)
            plt.subplot(i)
            plt.xlabel("index_value")
            plt.ylabel("value_count")
            plt.title(t)
            plt.bar(x, y)
        plt.show()

class WordProcess():
    """函数列表：获取文件列表get_filename、获取文件内容get_content、获取wiki语料"""
    #数据预处理部分
    def __init__(self,filein,fileout,model_file=None):
        self.filein=filein
        self.fileout=fileout
        self.stoplist = []
        self.model_file=model_file
    def make_path(self,filepath):
        if not os.path.exists(filepath):
            os.mkdir(filepath)
        return filepath


    def get_filename(self,fileDir):
        """获取文件夹下的文件路径和名称，仅一层"""
        filepath=[]
        if os.path.exists(fileDir):
            for filename in os.listdir(fileDir):
                filepath.append(os.path.join(fileDir,filename))
        return filepath
    def get_filecontent(self,filename):
        """文本内容转化为列表，一行一列"""
        #to [[],[]]
        data=[]
        with open(filename,'r',encoding='utf-8')as f:
            for line in f.readlines():
                data.append(line.strip(' ').strip('\n').strip('\t'))
        return data



    def merge_corpus(self,infilepath,output_file):
        """"将多个text文件合并为一个并输出,每个文本为一个单元"""
        data=[]
        with open(self.fileout+output_file+'/.txt','w',encoding='utf-8')as f:
            for filename in os.listdir(infilepath):
                if '.txt' in filename:
                    file_path = os.path.join(infilepath, filename)
                    t = []
                    with open(file_path, "r", encoding="utf-8")as fr:

                        for line in fr.readline():
                            line = line.strip('\n').strip('\t')
                            if len(line) > 0:
                                t.append(line)

                    s=' '.join(list(map(str,t)))
                    f.writelines(s+'\n')
                    data.append(s)
                else:
                    continue
        return data
    def write_matrix(self,filename,text):
        #text=[]
        #写入矩阵 [[],[]]

        with open(filename,'w',encoding='utf-8')as f:
            for l in text:
                f.write(' '.join(l))
                f.write('\n')
        return
    def write_content(self,filename,text):
        """[[(),()],[(),()]]"""
        with open(filename,'w',encoding='utf-8')as f:
            for e in text:
                for e1,e2 in e:
                    f.write(str(e1)+':'+str(e2)+' ')
                f.write('\n')

    def record_error(self, filename, errorcontent):
        with open(filename, 'a', encoding='utf-8')as f:
            f.write(errorcontent + '\n')

     #将下载下来的维基百科xml文件装化为txt文件
    def parse_wiki_corpora(self,name):
        wname=self.filein+name
        with open(self.fileout+'wiki_text','w',encoding='utf-8')as f:
            wiki = WikiCorpus(wname, lemmatize=False, dictionary={})
            for text in wiki.get_texts():
                f.write(' '.join(text) + "\n")
        return

    #中文分词部分
    #jieba分词
    def add_dict_jieba(self,user_dict):
        with open(self.fileout+user_dict,'r',encoding='utf-8')as f:
            word=f.readline()
            jieba.add_word(word.strip().strip('\n').strip('\t'))
    def add_jieba_word(self,word,fre):
        jieba.add_word(word,freq=fre)

    def cfenci(self,doc,name):
        """输入每行是一个文本"""
        new_doc=[]#[['',''],['','']]
        #https://zhuanlan.zhihu.com/p/53521380
        fileout=self.fileout+"cfenci/"
        self.make_path(fileout)
        with open(fileout+name+'.txt','w',encoding='utf-8')as f:
            for line in doc:
                word=[]
                words = jieba.cut(line, cut_all=False)
                for w in words:
                    if w not in self.stoplist:
                        word.append(w)
                new_doc.append(w)
                f.write(' '.join(word))
                f.write('\n')
        print("{}文档分词完毕".format(name))
        return new_doc

    def stopwords(self,stop_name):
        stoplist=[]
        with open(self.filein+stop_name,'r',encoding='utf-8')as f:
            for s in f.readlines():
                s=s.strip().strip('\n').strip('\t')
                stoplist.append(s)
            self.stoplist=stoplist
        return self.stoplist

    #繁转简体,去除无关内容
    def get_content_convert(self,doc):
        """Conversions 转换参数
        hk2s: Traditional Chinese (Hong Kong standard) to Simplified Chinese
        s2hk: Simplified Chinese to Traditional Chinese (Hong Kong standard)
        s2t: Simplified Chinese to Traditional Chinese
        s2tw: Simplified Chinese to Traditional Chinese (Taiwan standard)
        s2twp: Simplified Chinese to Traditional Chinese (Taiwan standard, with phrases)
        t2hk: Traditional Chinese to Traditional Chinese (Hong Kong standard)
        t2s: Traditional Chinese to Simplified Chinese
        t2tw: Traditional Chinese to Traditional Chinese (Taiwan standard)
        tw2s: Traditional Chinese (Taiwan standard) to Simplified Chinese
        tw2sp: Traditional Chinese (Taiwan standard) to Simplified Chinese (with phrases)"""
        text=[]
        #regex_str=re.compile("[^<doc.*>$]|[^</doc>$]")
        cc = opencc.OpenCC('t2s')
        with open(self.fileout+'opencc_convert.txt','w',encoding='utf-8')as f:
            for c in doc:
                c=c.strip('\n').strip('\t')
                # 移除空行，移除其他字符
                #regex_str.sub('',c)
                if len(c)>0:
                    c=cc.convert(c)
                    text.append(c)
                    f.write(c+'\n')
        return text
    #英文分词

    def word_cut(self,doc,name='efenci.txt'):
        # stemmerlan = LancasterStemmer()
        # stemmerporter = PorterStemmer()
        # lemmatizer = WordNetLemmatizer()
        stop_list=stopwords.words('english')
        #去除标点符号后分词和不去除标点符号分词有区别吗？应该是没有
        x = re.compile('[%s]' % re.escape(string.punctuation))  #去除标点符号

        text=[]
        for d in doc :
            #统一为小写.去除标点符号
            d= x.sub(u'', d.lower())
            #避免有中文符号的影响 尤其是“”‘’
            d = re.sub("[{}]+".format(punctuation), "", d)
            if d is not None:
                w = self.lemmatize_all(d)
                # 处理词形
                word = [s for s in w if s not in stop_list]
                # 处理词干，提取词干可能会有影响，慎重使用,如 apples 会被处理成appl
                # word = [stemmerlan.stem(s) for s in word]
                text.append(word)
            else:
                pass
        self.write_matrix(self.fileout+name,text)
        return text
    #提取词干
    def get_stem(self,doc):
        stemerporter=PorterStemmer()
        new_word=[]
        for d in doc:
            w=stemerporter.stem(d)
            new_word.append(w)
        return new_word
    #词形
    def lemmatize_all(self,sentence):
        wnl = WordNetLemmatizer()
        for word, tag in nltk.pos_tag(nltk.word_tokenize(sentence)):
            if tag.startswith('NN'):
                yield wnl.lemmatize(word, pos='n')
            elif tag.startswith('VB'):
                yield wnl.lemmatize(word, pos='v')
            elif tag.startswith('JJ'):
                yield wnl.lemmatize(word, pos='a')
            elif tag.startswith('R'):
                yield wnl.lemmatize(word, pos='r')
            else:
                pass

    def sentence_cut(self,doc):
        #划分句子
        text=sent_tokenize(doc)
        return text
    #向量化

    def word_bag(self,texts,fre=0):
        """texts格式
        [['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
        ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
        """
        # 去除低频词
        frequency = defaultdict(int)
        for text in texts:
            for token in text:
                frequency[token] += 1
        texts = [[token for token in text if frequency[token] > fre] for text in texts]
        #输出词频
        frequency=sorted(frequency.items(),reverse=True,key=lambda k:k[1])
        with open(self.fileout+'all_word_frequence.txt','w',encoding='utf-8')as f:
            json.dump(frequency,f,ensure_ascii=False,indent=4)
        # bag-of-words
        #pprint(texts[0:2])
        dictionary = corpora.Dictionary(texts)
        dictionary.save(self.fileout+'worddictionary.dict')#store the dictonary ,for future reference
        #输出词典词
        # pprint(dictionary.token2id)
        # 将其他文档转化为向量,如果新的文档中出现了不在词袋中的词，那么这些词将被舍弃
        # new_doc = 'Human computer interaction'
        # new_vec = dictionary.doc2bow(new_doc.lower().split())
        # print(new_vec)
        # 将原始的文档向量化
        corpus = [dictionary.doc2bow(text) for text in texts]
        #pprint(corpus)
        # 保存向量化后的文档
        corpora.MmCorpus.serialize(self.fileout+'corpus.mm', corpus)
        return (corpus,dictionary)

    def read_content(self,dict_file,corpus_file):
        print("读取词典数据和向量数据")
        # 词典数据
        dictionary = corpora.Dictionary.load(dict_file)
        # 向量数据
        corpus = corpora.MmCorpus(corpus_file)
        return (dictionary,corpus)

    def word_tfidf(self,corpus):
        tfidf = models.TfidfModel(corpus,normalize=True)  # initialize a model
        # transform new document,use the model to transform vectors
        # apply the model to a whole corpus,in this case,just use the same corpus ,
        # Once the transformation model has been initialized, it can be used on any vectors
        # (provided they come from the same vector space, of course), even if they were not used in the training corpus at all
        # Calling model[corpus] only creates a wrapper around the old corpus document stream – actual conversions are done on-the-fly

        corpus_tfidf = tfidf[corpus]
        corpus_tfidf.save(self.fileout+'model.tfidf')
        # when this done ,can save the results,see 语料库的保存
        # We cannot convert the entire corpus at the time of calling corpus_transformed = model[corpus],
        #for doc in corpus_tfidf:
            #print(doc)
        return corpus_tfidf
    def word_lsi(self,curpos_tfidf,dictionary,num_topic):
        # Transformations can also be serialized, one on top of another, in a sort of chain
        # Here we transformed our Tf-Idf corpus via Latent Semantic Indexing into a latent 2-D space (2-D because we set num_topics=2)
        lsi_model = models.LsiModel(curpos_tfidf, id2word=dictionary, num_topics=num_topic)
        lsi_model.save(self.fileout+'model.lsi')
        return lsi_model
    def word_lda(self,corpus,dictionary):
        lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)
        lda_model.save(self.fileout+'model.lda')
        return lda_model
    def train_word2vec_single(self,input_file,size=256, window=10, min_count=5):
        #一行为一个文本,可以直接为txt文件名
        sentence=LineSentence(input_file)
        #防止语料过大无法加载
        # 训练模型
        model=Word2Vec(sentence,size=size,window=window,min_count=min_count,workers=multiprocessing.cpu_count(),iter=10)
        model.save(self.fileout+'wiki_corpus.model')
        model.wv.save_word2vec_format(self.fileout+'word2vec_format',binary=False)
        return model
    def train_word2vec_dir(self,input_dir,size=256, window=10, min_count=5):


       # 输入语料目录:PathLineSentences(input_dir)，input_dir为语料目录，而不是文件名
       #embedding size:256 共现窗口大小:10 去除出现次数5以下的词,多线程运行,迭代10次
        model = Word2Vec(PathLineSentences(input_dir), size=size, window=window, min_count=min_count, workers=multiprocessing.cpu_count(), iter=10)

        model.save(self.fileout + 'wiki_corpus.model')
        model.wv.save_word2vec_format(self.fileout + 'word2vec_format', binary=False)
        return model
    # 一次性加载语料
    def train_word2vec_filter(self):
        return
    def get_word_word2vector(self,words,model,k=300):
        #需确保words是不重复的
        word_vector=defaultdict()
        i=0
        for w in words :
            if w in model.wv.vocab:
                word_vector[w]=model.wv[w]
            else:
                i+=1
                word_vector[w]=np.random.uniform(-0.25,0.25,k)
        print("未登录词数为{}".format(i))
        return word_vector
    """关于归一化：
       因为余弦值的范围是 [-1,+1] ，相似度计算时一般需要把值归一化到 [0,1]，一般通过如下方式：
       sim = 0.5 + 0.5 * cosθ
       若在欧氏距离公式中，取值范围会很大，一般通过如下方式归一化：
       sim = 1 / (1 + dist(X,Y))"""
    def vector_cosine(self,v1,v2):
        d1=np.asarray([v1])
        d2=np.asarray([v2])
        num=float(d1.dot(d2.T))
        denom=np.linalg.norm(d1)*np.linalg.norm(d2)
        cos=num/denom #余弦值
        sim=0.5+0.5*cos#根据皮尔逊相关系数归一化
        return sim
    def vector_euclidean(self,v1,v2):
        d1 = np.asarray([v1])
        d2 = np.asarray([v2])
        dist = np.linalg.norm(d1 - d2)
        sim = 1.0 / (1.0 + dist)  # 归一化
        return sim



    def get_text_bagvector(self,dictionary,curpus):
        """取出每个句子的词",not for word2vec
        curpus格式
        [[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
        [(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555)]]
        text格式：每行为词和词的频率
        [[(),(),()],[(),()]]"""
        text=[]
        dict=dictionary.token2id
        #dict格式：dict of(str,int)
        for d in curpus:
            doc=[]
            for w ,f in d:
                # 比较id，获得词的频率
                t=[k for (k,v) in dict.items() if v==w][0]
                doc.append((t,f))
            doc.sort(key=lambda x: x[1], reverse=True)  # 进行排序
            """"
            #取出规定长度的词
            if len(doc)<list_length:
                pass
            else:
                doc=doc[0:list_length]"""
            text.append(doc)
        self.write_content(self.fileout+'word_frequence.txt',text)
        return text

    def load_word2vec(self):
        #加载预训练模型，如果模型中没有这个词，应当如何处理?
        # model = models.Word2Vec.load(self.model_file)
        model_file=self.model_file
        model=models.KeyedVectors.load_word2vec_format(model_file,binary=True)
        return model
    def wsimilarity_top_word(self,model,word,top=10):
        #获取最相似的词
        sim_word=model.most_similar(word,topn=top)
        return sim_word
    def words_similarity(self,w1,w2,model):

        w1_new=[w for w in w1 if w in model.wv.vocab]
        w2_new=[w for w in w2 if w in model.wv.vocab]
        if len(w1)!=len(w1_new) or len(w2)!=len(w2_new):
            self.record_error(self.fileout+'not_in_word2vec_vocab.txt',"原句1：{},新句1:{};原句2:{},新句2{}".format(w1,w1_new,w2,w2_new))
        #比较两个词列表的相似性
        if len(w1_new)!=0 and len(w2_new)!=0:
            sim=model.wv.n_similarity(w1_new,w2_new)
        else:
            print(w1,w2)
            sim=0
        return sim

    def compare_word_similarity(self,model,word_list):
        word_pair_sim=[]
        with open(self.fileout+"word_pair_similarity.txt",'a',encoding='utf-8')as f:
            for b_word, e_word in word_list:
                res = model.similarity(b_word, e_word)
                w_p_s=set(b_word, e_word, res)
                word_pair_sim.append(w_p_s)
                f.write(w_p_s)
                f.write('\n')
        return word_pair_sim

class GraphProcess():
    """build and clean the Graph"""
    def  __init__(self,filein,fileout):
        self.filein=filein
        self.fileout=fileout

    def add_node_attribute(self,G,n_a,att_name):
        #n_a is a {node_id:att},if att is a dict ,then the key of att will be the attribute' name
        if isinstance(dict,n_a):
            nx.set_node_attributes(G,n_a,att_name)
            print("add attribute to node success")
        return G
    def add_edge_attribute(self,G,e_a,att_name):
        #e_a must be {(n1,n2):att} ,att is the same as before
        if isinstance(dict,e_a):
            nx.set_node_attributes(G,e_a,att_name)
            print("add attribute to edge success")
        return G

    #按照度清洗节点
    def isloated(self,G):
        G1=G.copy()
        #remove only degree-zero nodes
        #Note:if remove the node,please remove the node attribute at the same time
        s=list(nx.isolates(G1))
        G1.remove_nodes_from(s)
        skip_node=list(map(str,s))
        nx.write_gexf(G1, self.fileout + 'clean_0_Graph.gexf', encoding='utf-8')
        return G1, skip_node

    def prune(self,g, limit=2):
        G=g.copy()
        # remove nodes which have  too few neighbors
        skip=[]
        for v in list(G.nodes()):
            d = G.degree(v)
            if d < limit:
                skip.append(v)
        if len(skip) > 0:
            G.remove_nodes_from(skip)
            skip = list(map(str, skip))
            nx.write_gexf(G, self.fileout + 'clean_{}_Graph.gexf'.format(limit), encoding='utf-8')
        return G, skip
    def neighbor_hop(self,G,hop=0):
        neighbors=defaultdict(lambda :'N/A')
        #{id:[]}
        """definite the neighborhood ,hop means the range of neighbors
        hop=0: the neighbors as well as the node itself,0-1 hop
        hop=1:only the direct neighbors of the node  ,1 hop
        hop=2: neighbors of the neighbors and include the node itself ,2 hop"""
        for node in list(G.nodes()):
            nei=[]
            #包含0节点时节点自自己和自己的相似性为1，会放大自身标签传播的可能
            if hop == 0:
                # print("0-1 hop")
                nei = list(G.neighbors(node)) + [node]
            elif hop == 1:
                # print("1 hop")
                nei = list(G.neighbors(node))
            #与0-1 hop 同样的问题
            elif hop == 2:
                # print("2 hop")
                nei1 = list(G.neighbors(node))
                nei=nei1+[node]
                for node in nei1:
                    nei2 = list(G.neighbors(node))
                    nei=nei+nei2
            node=str(node)
            nei=list(map(str,nei))
            neighbors[node] = nei
        return neighbors
    def visual_G(self,G,c_G):
        print("绘制原始图及处理后图")
        plt.figure(figsize=(10,40),dpi=500)
        plt.subplot(411)
        nx.draw_networkx(G,nx.spring_layout(G))
        plt.axis('off')
        plt.title('Origin Graph')

        plt.subplot(412)
        d_g_o = nx.degree_histogram(G)
        d_g_o_s = pd.Series(d_g_o)
        d_g_o_s.to_csv(self.fileout + 'degree_distribution_graph.txt')
        x = np.arange(0, len(d_g_o), 1)
        plt.bar(x, d_g_o)
        plt.title('The degree distribution of origin graph')
        plt.xlim(0,len(x)+1)
        plt.ylim(0,max(d_g_o)+1)
        plt.xticks(np.linspace(0,len(x),num=10,endpoint=True))
        plt.yticks(np.linspace(0, max(d_g_o), num=10, endpoint=True))
        plt.xlabel('degree')
        plt.ylabel('degree count')

        print("原始图信息")
        print(nx.info((G)))
        print("原始图度分布")
        print(d_g_o)
        plt.subplot(413)
        print(nx.info(c_G))
        nx.draw_networkx(c_G,pos=nx.spring_layout(c_G))
        plt.axis('off')
        plt.title('Processed Graph')
        plt.subplot(414)
        d_g = nx.degree_histogram(c_G)
        d_g_s=pd.Series(d_g)
        d_g_s.to_csv(self.fileout+'degree_distribution_processed_graph.txt')
        x = np.arange(0, len(d_g), 1)
        plt.bar(x, d_g)
        plt.title('The degree distribution of processed graph')
        plt.xlim(0, len(x)+1)
        plt.ylim(0, max(d_g)+1)
        plt.xticks(np.linspace(0, len(x), num=10, endpoint=True))
        plt.yticks(np.linspace(0, max(d_g), num=10, endpoint=True))
        plt.xlabel('degree')
        plt.ylabel('degree count')

        print("删除部分节点后图信息")
        print(nx.info(c_G))
        print("删除部分节点后图度分布")
        print(d_g)
        plt.savefig(self.fileout+"Graph information.png",dpi=500)
        plt.show()
class SlpaExtend():
    """A program to identify the overlapping community and overapping node which combine
    the semantic content of the node and topology of the network.The target of the program is
     to get  communities with specific meaning
      Note: all the data is clean ,it dosen't need to process again"""
    def __init__(self,filein,fileout):
        """Parameter:
                Iteration:number of iterations
                Threshold :t in [0,1],if the probability of the label is less than t,remove it
                SimThreshold: if the similarity between two nodes is less than SimThreshold,The listener
                won't accept the label of the spreaker
                Attcoeff:attenuation coefficient of the label during iteration"""
        self.filein=filein
        self.fileout=fileout
        #存储label  {t:{node_id:[(label,pro)]}}
        self.node_memory={}
        # 存储id  {t:{node_id:[(node,pro)]}}
        self.accept_id={}
        #modularity  {t:m}
        self.modu={}
        self.node_similarity=defaultdict()
        self.com_count={}
        self.com_iter_nodecount=defaultdict()
        self.node_iter_comcount=defaultdict()

        #the data structure of node_memory :{iterid:{node_id:[(label,propority),()]},{}},iterid in [0,T]
        #origin_label  means the original label of nodes when initialize
        #previous_label means the label of node in the previous iteration
        #current_label represent the label of node in current iteration
        # self.config={'max_Iteration':Iteration,'similarity_Threshold':Sim_Threshold,\
        #              'hop':hop,'stop_condition':stop_condition,\
        #              'stop_Threshold':stop_Threshold,'cut_threshold':cut_threshold}
        """data structure
        adjacency_list:{id:[nei_id1,nei_id2]} for each node a list of its neighbours id
        node_memory:{id:[(label,probability)]} fro each node a list of label and label's probability
        """



    def get_node_label_matrix(self, node_label):
        label_matrix = []
        for label in node_label.values():
            label_matrix.append(label)
        return label_matrix

    def get_node_content(self,Idcontent_file):
        # 默认文件格式是id:content,建立节点的id,content对照表
        node_content =OrderedDict()
        with open(Idcontent_file, 'r',encoding='utf-8')as f:
            for c in f.readlines():
                c=c.strip().strip('\n').strip('\t').split(":")
                #防止出现多个冒号
                content=' '.join(c[1:len(c)])
                id=str(c[0])
                node_content[id]=content
        return (node_content)
    def prune_node_content(self,skip_node,node_content):
        del_node_content={}
        if skip_node is not None:
            #防止int 不能识别

            for id in skip_node:
                del_node_content[id]=node_content[id]
                del node_content[id]
            self.write_json_file(del_node_content,self.fileout+'drop_node_content.json')
        else:
            pass

        return node_content
    def get_node_label(self,i,id):
        label=defaultdict()
        for l,p in self.node_memory[i][id]:
            label.append(l)
        return label

    def get_node_content_list(self, node_content):
        content_list = []
        for content in node_content.values():
            content_list.append(content)
        return content_list
    def get_doc_word(self,wordprocess,node_content,Languagetag=0):
        """"处理未分词的文本
        node_content的形式是{node_id:''}"""
        content = self.get_node_content_list(node_content)
        if Languagetag == 0:
            # 英文分词
            doc = wordprocess.word_cut(content)
        else:
            # 中文繁体转简体
            doc1 = wordprocess.get_content_convert(content)
            # 中文分词
            doc = wordprocess.cfenci(doc1)
        return doc
    def get_doc_word_done(self,node_content):
        """"将已经分过词的句子转化为[['',''],['','']]"""
        doc=[]
        for content in node_content.values():
            c=content.split(' ')
            doc.append(c)
        return doc

    def initialize_label(self,wordprocess,node_content,Languagetag=0):
        """默认处理英文，Languagetag=0;处理中文时Languagetag=1

        初始化标签时对标签的分布做了标准化，即p/sum(p),转化为标签的初始概率分布
        再之后的迭代中每一轮都进行标准化吗？"""
        #node_label格式 {id:[]}
        #初始化标签
        #origin_label格式：{id:[(),()]}
        origin_label={}
        accept_id={}
        #对于未分词的原始文档，进行如下处理
        # print(node_content)
        doc=self.get_doc_word(wordprocess,node_content,Languagetag)
        # 对于已经分过词的，进行如下处理
        # doc=self.get_doc_word_done(node_content)
        # 转换为词袋
        curpus,dictionary=wordprocess.word_bag(doc)
        #转换为tf-idf
        corpus_tfidf=wordprocess.word_tfidf(curpus)
        #
        word_text=wordprocess.get_text_bagvector(dictionary,corpus_tfidf)

        for id, word_fre in zip(node_content.keys(),word_text):
            origin_label[id] = word_fre
            accept_id[id]=[(id,1)]
        filepath = self.make_filedir(self.fileout + 'original_label/')
        self.write_nodel_label(origin_label,filepath,'origin_label')
        origin_label_norm=self.label_frequency_normalize(origin_label)
        # origin_label_norm=origin_label

        self.write_nodel_label(origin_label_norm, filepath,'origin_label_normal')
        self.node_memory[0] =origin_label_norm
        self.accept_id[0]=accept_id
        return origin_label_norm,accept_id
    def label_frequency_normalize(self,n_label):
        #n_label  {id:[(id,pre)]}
        #需要优化数据结构
        node_label={}
        for id, word_fre in n_label.items():
            # print(word_fre)
            f_sum = sum([p for l,p in word_fre])
            sen: List[Tuple[Any, Union[float, Any]]] = []
            for label, fre in word_fre:
                sf=fre/f_sum
                sen.append((label, sf))
            sen=sorted(sen,key=lambda x: x[1], reverse=True)  # 进行排序
            node_label[id] = sen
            #如果出现节点的label为空怎么处理？
        return node_label
    def cut_label(self,n_label,cut_thre):
        # 如果选择过滤，度越大的节点接收到的标签种类越多，每个标签的fre较低，会被全部过滤
        # 如果sf小于一定值会被过滤掉，剩下的值之和不为1,还需要二次处理使得节点所属标签值和为1.需要两次归一化?
        node_label = defaultdict()
        #取一个固定的cut阈值
        for id, word_fre in n_label.items():
            sen=[(label,fre) for label ,fre in word_fre if fre>= cut_thre]
            #如果没有满足的，取最大的一个
            if len(sen)==0:
                sen=[sorted(word_fre,key=lambda x:x[1],reverse=True)[0]]
            sen=sorted(sen,key=lambda x: x[1], reverse=True)  # 进行排序
            node_label[id] = sen
            # 如果出现节点的label为空怎么处理？

        return node_label
    def cut_label_change(self,n_label,per=25):
        node_label = defaultdict()
        # 根据节点拓扑结构取cut阈值
        #暂定为四分位数

        for id, word_fre in n_label.items():
            cut_threshold=np.percentile([ fre for label, fre in word_fre],per)
            sen = [(label, fre) for label, fre in word_fre if fre >= cut_threshold]
            # 如果没有满足的，取最大的一个,在这里其实没有必要
            if len(sen) == 0:
                sen = [sorted(word_fre, key=lambda x: x[1], reverse=True)[0]]
            sorted(sen,key=lambda x: x[1], reverse=True)  # 进行排序
            node_label[id] = sen
            # 如果出现节点的label为空怎么处理？

        return node_label

    def cut_model(self, n_label_pro, n_accept_id, c_model=1, per=25,cut_threshold=0.05):
        print("cut_threshold:{}".format(cut_threshold))
        # 先标准化再切割？
        new_label_pro=n_label_pro
        new_accept_id=n_accept_id
        node_memory={}
        accept_id={}
        if c_model == 0:
            # 只归一化，不做其他处理
            node_memory = self.label_frequency_normalize(new_label_pro)
            accept_id = self.label_frequency_normalize(new_accept_id)
        elif c_model == 1:
            # 对所有标签按照百分位切割，然后再归一化
            new_label_pro = self.cut_label_change(new_label_pro, per)
            node_memory = self.label_frequency_normalize(new_label_pro)
            accept_id = self.label_frequency_normalize(new_accept_id)
        elif c_model == 2:
            # （2）对所有标签归一化，再按照固定的阈值切割后再归一化
            new_label_pro = self.label_frequency_normalize(new_label_pro)
            node_memory = self.cut_label(new_label_pro,cut_threshold)
            node_memory = self.label_frequency_normalize(node_memory)
            accept_id = self.label_frequency_normalize(new_accept_id)
        elif c_model == 3:
            # 对节点和所有标签归一化，再按照固定的阈值切割
            new_label_pro = self.label_frequency_normalize(new_label_pro)
            node_memory = self.cut_label(new_label_pro,cut_threshold)
            node_memory = self.label_frequency_normalize(node_memory)
            new_accept_id = self.label_frequency_normalize(new_accept_id)
            accept_id = self.cut_label(new_accept_id,cut_threshold)
            accept_id = self.label_frequency_normalize(accept_id)
        elif c_model == 4:
            # 对节点和所有标签按照百分位切割，然后再归一化
            new_label_pro = self.cut_label_change(new_label_pro, per)
            new_accept_id = self.cut_label_change(new_accept_id, per)
            node_memory = self.label_frequency_normalize(new_label_pro)
            accept_id = self.label_frequency_normalize(new_accept_id)

        return (node_memory, accept_id)

    def make_filedir(self,filepath):
        if not os.path.exists(filepath):
            os.mkdir(filepath)
        return filepath

    def write_nodel_label(self,node_label,filepath,filename):
        with open(filepath+filename+'.txt','w',encoding='utf-8')as f:
            for id ,label_fre in node_label.items():
                # print(id)
                f.write(str(id)+' ')
                for label ,fre in label_fre:
                    f.write(str(label)+':'+str(fre)+' ')
                f.write('\n')
        return

    def write_dict_list(self,d_list,filename):
        with open(filename,'w',encoding='utf-8')as f:
            for k ,v in d_list.items():
                f.write(str(k)+':'+' '.join(v))
                f.write('\n')
        return

    def stop_condition_stable(self,l,tag_com=None,stop_condition=1,stop_Threshold=0.0001):
        #l 是循环次数
        """stop the iteration when satisfy the condition
        'stop_condition' means the condition_form
        'stop_condition': the labels remain stable   how to compare the previous label and current label ?
        'stop_condition': the change of moudularity is lower than the threshold
        'stop_condition': the change of label similarity is lower than the threshold
        """
        if stop_condition == 0:
            diff = 0
            for node in tag_com.keys():
                #完全相同还是仅仅标签相同？
                if tag_com[l+1][node] != tag_com[l][node]:
                    diff += 1
            changeproportion = diff / self.node_count
            if changeproportion < stop_Threshold:
                return False
            else:
                return True
        elif stop_condition == 1:
            m=self.modu[l+1] - self.modu[l]
            if abs(m) < stop_Threshold:
                modu=sorted(self.modu.items() ,reverse=True,key= lambda k:k[1])
                print("排序后的模块度值")
                print(modu)
                return False
            else:
                return True


    def label_propagation(self,filepath,G,neighbor,node_id_memory,node_label_memory,wordprocess,model,order_list,sim_threshold,run_mode=1,c_model=1,per=25,cut_threshold=0.05):
        # order_list,节点排序,可根据需要改变
        # print("开始迭代循环")
        # 迭代循环
        self.node_count=len(order_list)
        file_label = self.make_filedir(filepath + 'node_label/')
        file_id = self.make_filedir(filepath + 'node_id/')
        file_com = self.make_filedir(filepath + 'community/')
        file_drop=self.make_filedir(filepath + 'drop_nest_community/')
        self.node_memory[0] = node_label_memory
        self.accept_id[0] = node_id_memory
        for t in range(100):
            # 异步更新
            print('开始第{}次迭代'.format(t+1))

            for listener in order_list:
                nei=[]
                w_listener=[]
                w_speaker=[]
                listener=str(listener)
                # print(listener)
                label_list = {}
                id_list = {}
                nei= neighbor[listener]
                n_listener=node_label_memory[listener]
                w_listener = [l for l,p in n_listener]
                # print(node_label_memory[listener])
                # print(node_label_memory)
                c = 0
                for speaker in nei:
                    speaker=str(speaker)
                    w_speaker=[]
                    # print(speaker)

                    nlm=node_label_memory[speaker]
                    # print(nlm)
                    w_speaker=[k for k ,v in nlm]
                    # print(w_speaker)

                    l_s = wordprocess.words_similarity(w_listener,w_speaker,model)

                    if l_s >= sim_threshold:
                        c+=1
                        for slabel, prob in nlm:
                            # add_pro = prob * l_s
                            add_pro = prob
                            if slabel in label_list:
                                label_list[slabel] += add_pro
                            else:
                                label_list[slabel] = add_pro
                        for id, fre in node_id_memory[speaker]:
                            # add_fre = fre * l_s
                            add_fre = fre
                            if id in id_list:
                                id_list[id] += add_fre
                            else:
                                id_list[id] = add_fre
                    else:
                            pass

                if c==0:
                    continue
                else:
                    label_list = sorted(label_list.items(), reverse=True, key=lambda k: k[1])
                    id_list = sorted(id_list.items(), reverse=True, key=lambda k: k[1])
                    #如果还出现重复，实际上是随机选择了
                    l_max = label_list[0]
                    i_max = id_list[0]
                    #获取listener当前标签,注意dict之后会改变顺序
                    listen_node_label=dict(n_listener)
                    listen_id_label=dict(node_id_memory[listener])

                    listen_node_label[l_max[0]]=listen_node_label.get(l_max[0],0)+l_max[1]
                    listen_id_label[i_max[0]] = listen_id_label.get(i_max[0], 0) + i_max[1]

                    node_label_memory[listener] = list(listen_node_label.items())
                    node_id_memory[listener] = list(listen_id_label.items())

            #如果需要截断，使用cut_model()
            # node_label_memory=self.label_frequency_normalize(node_label_memory)
            # node_id_memory=self.label_frequency_normalize(node_id_memory)
            node_label_memory,node_id_memory=self.cut_model(node_label_memory,node_id_memory,c_model,per,cut_threshold)
            self.node_memory[t + 1] = node_label_memory
            self.accept_id[t+1]=node_id_memory

            self.write_nodel_label(self.node_memory[t + 1],file_label, 'node_label_'+str(t + 1))
            self.write_nodel_label(self.accept_id[t + 1],file_id, 'node_id_'+str(t + 1))
            #partition  社团划分结果，NodeClustering类，comm  {com_id:[]}
            #new_nodel_label  规定了节点所属的社团数后截断得到的node_label  {node_id:[(label,pro)]}
            #comm_size   每个社团的节点个数，{com_id:node_count}
            #node_com_size  每个节点的社团个数  {node_id:com_count}
            if run_mode == 1:

                # 以节点序号为社团标签
                # #如果使用非重叠模块度，那get_community中的最后一个参数c必须选1
                # partition, comm, new_node_label, comm_size, node_com_size = self.get_community(t+1,self.accept_id[t + 1], 1)
                # self.modu[t + 1] = ev.newman_girvan_modularity(self.G,partition).score
                # print(self.modu[t + 1])

                # 如果使用重叠模块度,c='all'或者规定的节点最多社团数，这个时候需要配合cut_model使用

                partition, comm, node_com_label,node_com_size = self.get_community(G,t + 1,node_id_memory,file_com, 'all')

            else:
                # 以节点标签为社团标签
                # 如果使用非重叠模块度，那get_community中的最后一个参数c必须选1
                # partition, comm, new_node_label, comm_size, node_com_size = self.get_community(t + 1, self.node_memory[t + 1], 1)
                #
                # self.modu[t + 1] = ev.newman_girvan_modularity(self.G, partition).score
                # print(self.modu[t + 1])
                # 如果使用重叠模块度,c='all'或者规定的节点最多社团数，这个时候需要配合cut_model使用

                partition, comm, node_com_label, node_com_size = self.get_community(G,t + 1,node_label_memory,file_com, 'all')
            partition, comm, nest_com, comm_size = self.drop_nested_community(G,file_drop,comm,t+1)
            new_node_com_label, nest_node, node_com_size,node_comm = self.drop_nested_node_community(file_drop,nest_com, node_com_label,t+1)

            modu = ev.link_modularity(G, partition).score
            self.modu[t + 1] = modu
            #每个社团的大小

            c_s= len(comm_size.keys())
            self.com_count[t + 1] =c_s
            self.com_iter_nodecount[t + 1] = comm_size
            self.node_iter_comcount[t + 1] = node_com_size
            print("社团数量{}".format(c_s))
            print("模块度为{}".format(modu))
            if t>0:
                #注意，如果需要第一种停止方式：节点的tag相同，则需要传入第二个参数给stop_condition_stable(t,tag_com)
                if self.stop_condition_stable(t,tag_com=self.node_memory,stop_condition=1,stop_Threshold=0.001):
                    print("The {} time loop end".format(t+1))
                    continue
                else:
                    print("The loop is terminated because the condition is satisfied or the loop  have get the max_iteration.the current loop time is {}".format(t+1))
                    break
        # 以下的new_node_com_label 实际上是节点的社团归属，node_id或是label_id取决于run_model
        #需要注意的是new_node_com_label 是没有归一化的，因为节点标签已经没有办法根据删去的社团找到，因此残留了标签？
        #实际上这里是不必要的，需要取最后一次即可

        com_label = self.get_community_label(file_com,comm, node_label_memory)
        self.write_dict_list(comm,filepath+'final_community.txt')
        self.write_dict_list(node_comm, filepath + 'final_node_to_community.txt')
        readwrite.write_community_csv(partition,filepath+'cdlib_final_community.csv')
        self.write_json_file(self.com_count, filepath + 'com_count.txt')
        self.write_json_file(self.com_iter_nodecount, filepath + 'com_iter_nodecount.txt')
        self.write_json_file(self.node_iter_comcount, filepath + 'node_iter_comcount.txt')
        self.write_json_file(self.modu, filepath + 'modularity.txt')
        self.write_json_file(self.accept_id, filepath + 'accept_id.txt')
        self.write_json_file(self.node_memory, filepath + 'node_label.txt')
        # self.write_json_file(self.node_similarity, self.fileout + 'node_similarity.txt')
        print("删除的社团数量{},受到影响的节点数量{}".format(len(nest_com), len(nest_node)))
        print('最终的社团数量{}'.format(c_s))
        print("最终的模块度{}".format(modu))
        conduance=ev.conductance(G,partition).score
        print("电导率为{}".format(conduance))

        return partition, com_label,modu,conduance,c_s

    def Emodularity(self, G, com_node, neigh, node_com):
        #com_node: {com:[node_id]}, neigh:  {node_id:[node_id,node_id]}, node_com  {node_id:[com_id]}

        com = com_node.keys()
        m = float(G.number_of_edges())
        score = 0
        error_node = 0

        for c in com:
            for v in com_node[c]:
                  for u in com_node[c]:
                    if v == u:
                        pass
                    else:
                        try:
                            if u in neigh[v]:
                                a = 1
                            else:
                                a = 0
                            ov = len(node_com[v])
                            ou = len(node_com[u])
                            kv = len(neigh[v])
                            ku = len(neigh[u])
                            score = score + (1 / (ov * ou) * (a - (kv * ku) / (2 * m)))
                        except Exception as e:
                            print("节点{}错误".format(v))
                            error_node = error_node + 1
                            continue
                        else:
                            pass
        print("错误节点数{}".format(error_node))
        score = float(score / (2 * m))
        return score

    def write_json_file(self,data,filename):
        f=open(filename,'w',encoding='utf-8')
        json.dump(data,f,ensure_ascii=False,indent=4)
    def drop_nested_community(self,G,file,c,t):
        communities=c.copy()
        #communities   {com_id:[node_id,,]}
        # Remove nested communities
        nested_communities = set()
        keys = list(communities.keys())
        for i, label0 in enumerate(keys[:-1]):
            comm0 = set(communities[label0])
            for label1 in keys[i + 1:]:
                comm1 = set(communities[label1])
                if comm0.issubset(comm1):
                    nested_communities.add(label0)
                elif comm0.issuperset(comm1):
                    nested_communities.add(label1)

        for comm in nested_communities:
            del communities[comm]
        n_c=pd.Series(list(nested_communities))
        n_c.to_csv(file+'nested_communities_{}.csv'.format(t),index=False)
        com=list(communities.values())
        partition=NodeClustering(com, G, "SLPA", method_parameters={"T": 100, "r": 0.25}, overlap=True)

        self.write_dict_list(communities,file+'communities_node_{}.txt'.format(t))
        com_size={com:len(node_list) for com ,node_list in communities.items()}
        return partition,communities,nested_communities,com_size
    def drop_nested_node_community(self,file,nested_com,n_label,t):
        node_label=n_label.copy()
        # node_label   {node_id:[(node_id,pro)]}
        #nested_com    []需要删除的列表
        com_node = defaultdict()
        new_node_label={}
        #统计受到影响的节点
        nest_node={}
        node_com_size={}
        node_comm={}
        for k, v in node_label.items():
            label_pro=[]
            l=[]
            i=0
            for label, pre in v:
                if label in nested_com:
                    i=i+1
                    continue
                else:
                    label_pro.append((label,pre))
                    l.append(label)
            nest_node[k]=i
            label_pro=sorted(label_pro,reverse=True,key= lambda k:k[1])
            new_node_label[k]=label_pro
            node_com_size[k]=len(label_pro)
            node_comm[k]=l
        # new_node_label=self.label_frequency_normalize(new_node_label)
        n_cover = [k for k, v in node_label.items() if len(v) == 0]
        print("无归属社区节点有{}个".format(len(n_cover)))
        n_cover=pd.Series(n_cover)
        n_cover.to_csv(file+'no_community_node_{}.csv'.format(t),index=False)

        n_n=pd.Series(list(nest_node))
        n_n.to_csv(file+'nest_node.csv',index=True)
        self.write_dict_list(node_comm,file+'node_communities_{}.txt'.format(t))
        return new_node_label,nest_node,node_com_size,node_comm



    def get_community(self,G,t,node_label,filepath,node_com_count='all'):
        #node_label  {node_id:[(label,pro)]}
        #输出{com_id:[node_id]}
        comm=defaultdict()

        node_com_size={}
        new_node_label ={}
        for id ,l_pro in node_label.items():
            label_pro=sorted(l_pro,reverse=True,key= lambda k:k[1])
            #确定节点所属社团个数
            if node_com_count =='all':
                pass
            elif node_com_count==1:
                label_pro=l_pro[0]
            else:
                label_pro=[(l,p) for l ,p in l_pro if p >node_com_count]
            #计算节点所属社团个数
            node_com_size[id]=len(label_pro)
            #记录新的节点社团标签
            new_node_label[id]=label_pro
            #转化为{com:[node_id]}
            for label ,pro in label_pro:
                if label in comm.keys():
                    comm[label]+=[id]
                else:
                    comm[label]=[id]

        self.write_dict_list(comm,filepath+'{}iter_community.txt'.format(t))
        comm_list = list(comm.values())
        #注意返回的是cdlib中的类，方便下一步直接调用
        communities = NodeClustering(comm_list, G, "SLPA", method_parameters={"T": 100, "r": 0.25}, overlap=True)
        # readwrite.write_community_csv(communities, fileout + '{}_gold_community.csv'.format(g_name))
        return (communities,comm,new_node_label,node_com_size)
    def get_community_label(self,file,comm,node_label):
        #获取社团标签
        #com_label  {com:{label:fre}}
        com_label={}
        # print(comm)
        for com ,node in comm.items():
            #计数
            label_list=[]
            for n in node:
                l_list=[l for l ,p in node_label[n]]
                label_list.extend(l_list)
            com_label[com]=dict(Counter(label_list))
        self.write_json_file(com_label,file+'com_label.txt')
        return com_label
    def label_wordcloud(self,fileout,com_lable,image_file):
        #com_label  {com:{label:fre}}

        for com ,label in com_lable.items():
            # print(label)

            # read the mask image
            # taken from
            # http://www.stencilry.org/stencils/movies/alice%20in%20wonderland/255fk.jpg
            alice_mask = np.array(Image.open(image_file))
            stopwords = set(STOPWORDS)
            # stopwords.add("said")
            wc = WordCloud(background_color="white", max_words=2000, mask=alice_mask,
                           stopwords=stopwords, contour_width=3, contour_color='steelblue')
            # generate word cloud
            wc.generate_from_frequencies(label)

            # store to file
            wc.to_file(fileout+'{}_wordcloud.png'.format(com))
            # show
            plt.imshow(wc, interpolation='bilinear')
            plt.axis("off")
            plt.figure()
            plt.imshow(alice_mask, cmap=plt.cm.gray, interpolation='bilinear')
            plt.axis("off")
            plt.show()

    def Plot(self,file,data,title,xlabel,ylabel):
        s=plt.figure(figsize=(10,10),dpi=500)
        x=list(data.keys())
        y=list(data.values())
        plt.plot(x,y)
        plt.title(title)
        plt.xlim(0, len(x) + 1)
        plt.ylim(0, max(y) + min(y))
        plt.xticks(np.linspace(0, len(x), num=10, endpoint=True))
        plt.yticks(np.linspace(0, max(y), num=10, endpoint=True))
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.savefig(file+title+'.png',dpi=500)
        plt.show()

def make_filedir(filepath):
    if not os.path.exists(filepath):
        os.mkdir(filepath)
    return filepath
def slpa_main(graph_name):
    # 初始化输入信息

    # 输入文件
    # filein = 'D:/Backup Plus/slpa-dataset/slpa_compare_input/'
    filein='F:/aps_out/aps/APS_network/aps_article_network/article_journal_network/'
    filein1= 'D:/Backup Plus/slpa-dataset/slpa_compare_input/'
    # Word2vec模型文件
    model_file = filein1 + 'GoogleNews-vectors-negative300.bin'
    # 词云背景图片
    image_file = filein + 'logo.png'
    # 文件
    # graph_name = 'graph_pub_all_asyn_simcut3_simpre/'
    file_out = make_filedir('F:/slpa_aps_cut2pre/' + graph_name)
    file_graph = make_filedir(file_out + '/graph_process/')
    file_word = make_filedir(file_out + '/word_process/')
    # 图边信息,需要和下面的节点内容文件对应
    G_file = filein + graph_name+'.gexf'
    # 节点内容文件
    Idcontent_file = filein +graph_name+ '.txt'

    # 设定算法阈值
    # 最大循环次数，默认100
    # 节点相似度阈值
    Sim_Thresholds = [0, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    # Sim_Thresholds = [0.1, 0.2]
    # 停止条件,0:节点标签不再变动；1：模块度变化的绝对值小于阈值；2：未实现
    stop_conditions = [0, 1, 2]
    # stop_Thresholds = [0.0001, 0.01]
    # 邻居节点范围，0： 0-1 hop;1；1 hop,2 :2 hop
    hops = [0, 1, 2]
    stop_Threshold = [0.0001, 0.001]
    cut_threshold = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4,
                     0.45, 0.5]

    # 初始化类
    g = GraphProcess(filein, file_graph)
    wordprocess = WordProcess(filein, file_word, model_file)
    # 加载模型
    model = wordprocess.load_word2vec()
    print("处理图{}".format(graph_name))
    # 读取图
    G = nx.read_gexf(G_file, node_type=str)
    # 判断图的类型
    if nx.is_directed(G):
        G = G.to_undirected()

    # 去除度为零的节点
    c_G, skip_node = g.isloated(G)
    # 若需要可去除度小于阈值的节点
    # c_G, skip_node = g.prune(G)
    g.visual_G(G, c_G)
    # 获取邻居节点,hop=0:0-1 hop ;1: 1 hop;2: 2-hop
    neighbors = g.neighbor_hop(c_G, 1)
    # compare_algorithm_crisp(c_G,filein+'crisp.xlsx')
    # compare_algorithm_overlap(c_G,filein+'overlap.xlsx')
    # c_G,skip_node=g.prune(G,limit=1)
    # 初始化节点更新类
    o = NodeOrder(c_G)
    # 节点排序,返回排序结果和同一指标值的统计
    # order_list,c=o.degree_order()
    # o1 = pd.Series(order_list)
    # print("保存节点排序结果")
    # o1.to_csv(file_graph + 'order_list_.csv', index=False)
    # c1=pd.Series(c)
    # c1.to_excel(file_graph+'order_count_.xlsx',index=True)

    # 初始化slpa_extend
    file_slpa = make_filedir(file_out + '/slpa_perform/')
    slpa = SlpaExtend(filein, file_slpa)
    # 初始化标签
    # #获取节点内容
    node_content = slpa.get_node_content(Idcontent_file)
    p_node_content = slpa.prune_node_content(skip_node, node_content)
    print("原始节点个数为{},保存节点个数为{},删去节点{}个".format(len(node_content), len(p_node_content),
                                               len(p_node_content) - len(node_content)))

    node_label, node_id = slpa.initialize_label(wordprocess, p_node_content)
    # 循环次数
    iter_all = 3
    iter_cal = {}
    file_plot = make_filedir(file_slpa + 'plot/')
    for v in Sim_Thresholds:
        for c in cut_threshold:

            for t in range(iter_all):
                cal = {}
                f_n = str(v) + '_' + str(c) + '_' + str(t)
                file_propagation = make_filedir(file_slpa + f_n + '/')
                # 随机节点排序时
                order_list = o.random_order()
                o1 = pd.Series(order_list)
                print("保存节点排序结果")
                o1.to_csv(file_graph + 'order_list_{}.csv'.format(t), index=False)
                print("SLPA算法第{}次循环".format(t))
                # partition ：cdlib的nodeclustering ,com_label:社团  {com:[node]};modu:模块度
                # 注意变量在此处赋值给某一个参数,由于选了cut 模式，因此c_model只能选择2或者3
                node_id_memory = node_id.copy()
                node_label_memory = node_label.copy()
                st = time.time()
                partition, com_lable, modu, conduance, c_s = slpa.label_propagation(file_propagation, c_G, neighbors,
                                                                                    node_id_memory, node_label_memory,
                                                                                    wordprocess, model, order_list,
                                                                                    sim_threshold=v, run_mode=1,
                                                                                    c_model=2,
                                                                                    per=25, cut_threshold=c)
                # readwrite.write_community_csv(partition, file_slpa + 'cdlib_final_community.csv')
                sp = time.time()
                cal['modularity'] = modu
                cal['conduance'] = conduance
                cal['comcount'] = c_s
                cal['time'] = sp - st
                slpa.Plot(file_plot, slpa.modu, "modularity_{}".format(f_n), xlabel='iter', ylabel='modularuty')
                slpa.Plot(file_plot, slpa.com_count, 'community_count_{}'.format(f_n), 'iter', 'community size')
                try:
                    cal['newman_modularity'] = ev.newman_girvan_modularity(c_G, partition).score
                except:
                    print("newman modularity failed")

                cal['conduance'] = ev.conductance(c_G, partition).score
                iter_cal[f_n] = cal
                # 避免绘图后出现图的节点改变的问题
                # G_p = c_G.copy()
                # plt.figure()
                # viz.plot_network_clusters(G_p, partition, position=nx.spring_layout(c_G), figsize=(10, 10),
                #                           plot_overlaps=True)
                # plt.savefig(file_plot + "network_clusters_{}.png".format(f_n), dpi=500)
                #
                # plt.figure()
                # viz.plot_community_graph(G_p, partition, figsize=(10, 10), plot_overlaps=True)
                # plt.savefig(file_plot + "network_community_{}.png".format(f_n), dpi=500)
                # # plt.savefig(file_slpa + "network_community.eps", format='eps',dpi=500)
                # file_wordcloud=make_filedir(file_propagation+'wordcloud/')
                # slpa.label_wordcloud(file_wordcloud,com_lable, image_file)
                # plt.show()
                # 显式置空类的属性
                slpa.accept_id = {}
                slpa.node_memory = {}
                slpa.modu = {}
                slpa.com_count = {}
                slpa.com_iter_nodecount = {}
                slpa.node_iter_comcount = {}

    dt = pd.DataFrame(iter_cal)
    dt1 = dt.stack()
    dt2 = dt1.unstack(0)
    dt2.to_excel(file_slpa + 'calculate_output.xlsx', index=True, header=True)


if __name__ == '__main__':
    #'graph-16', 'graph-24','graph-75','graph_pub_all' ,
    # g_file1 = ['graph-107', 'graph-131', 'graph-144', 'graph-145', 'graph-162', 'graph-182', 'graph-199' ]
    # 'PRI',
    g_file1 = [ 'PRSTPER','RMP']
    for graph_name in g_file1:
        slpa_main(graph_name)